imports

pudl set up

In [13]:
# import the necessary packages
%load_ext autoreload
%autoreload 2

import logging
import os
import sys

import pandas as pd
import sqlalchemy as sa
import pudl

In [14]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)


read excel file from online

link: https://screeningtool.geoplatform.gov/en/downloads#3/33.47/-97.5

In [2]:
communities = pd.read_excel('https://static-data-screeningtool.geoplatform.gov/data-versions/1.0/data/score/downloadable/1.0-communities.xlsx')

geoprocessing prep

In [32]:
# get county ID
#make column a string
communities['Census tract 2010 ID'] = communities['Census tract 2010 ID'].astype(str)
# make sure census tract ID is 11 digits
communities['Census tract 2010 ID'] = communities['Census tract 2010 ID'].str.zfill(11)

#make county fips column
communities['county_id_fips'] = communities['Census tract 2010 ID'].str[0:5]

filter to disadvantaged

In [33]:
disadvantaged_communities = communities.loc[communities['Identified as disadvantaged']==True]

count how many tracts are in each county

In [34]:
num_of_dc = disadvantaged_communities['county_id_fips'].value_counts().reset_index()
num_of_dc = num_of_dc.rename(columns={'index':'county_id_fips','county_id_fips':'num_of_communities'})

connect to total amount

In [36]:
total_dc = communities['county_id_fips'].value_counts().reset_index()
total_dc = total_dc.rename(columns={'index':'county_id_fips','county_id_fips':'total_num_of_communities'})

In [40]:
community_complete = num_of_dc.merge(total_dc,on='county_id_fips',how='left')

In [42]:
community_complete['disadvantaged_percent_of_total_communities'] = community_complete['num_of_communities'] / community_complete['total_num_of_communities'] * 100
community_complete

,county_id_fips,num_of_communities,total_num_of_communities,disadvantaged_percent_of_total_communities
0,06037,1146,2346,48.849105
1,17031,611,1319,46.322972
2,36047,410,761,53.876478
3,48201,402,786,51.145038
4,26163,362,611,59.247136
...,...,...,...,...
2707,31151,1,4,25.000000
2708,31153,1,43,2.325581
2709,31161,1,2,50.000000
2710,31181,1,2,50.000000


get utility service territory from PUDL

In [46]:
eia_service_territory = pudl_out.service_territory_eia861()

In [48]:
eia_service_territory.groupby(['state','utility_id_eia'])